<div dir="rtl">

با سلام و عرض وقت،

ممنونم که این تکلیف را بررسی می‌کنید.

با عرض پوزش، داشبورد سازمان غذا و دارو از دسترس خارج بود و من یک فایل جایگزین پیدا کردم و مراحل را پیش رفتم. به همین دلیل سوالات ۶ تا ۱۰ بدون پاسخ ماند.

در مورد بقیه وظایف، بیش از آنچه فکر می‌کردم زمان برد و متاسفانه به مشکل تبدیل داده‌ها به دیتابیس PostgreSQL برخوردم.

به همین منظور، تا شنبه آخر وقت، هر آنچه در توانم بود و زمان داشتم را درست کردم و برایتان می‌فرستم. اگر فرصت شد، بقیه موارد را هم تکمیل خواهم کرد.

</div>


# تحلیل سالانه فروش دارویی

در این بخش، داده‌های فروش دارویی برای سال‌های **۱۴۰۱ و ۱۴۰۲** آماده شده است.

### داده‌ها شامل:
- `sales_value`: ارزش ریالی کل فروش هر سال
- `sales_volume`: تعداد کل واحدهای تامین شده هر سال
- `value_growth_rate`: نرخ رشد ارزش فروش نسبت به سال قبل (درصد)
- `volume_growth_rate`: نرخ رشد حجم فروش نسبت به سال قبل (درصد)


In [9]:
import pandas as pd
pharma_statistics = (pd.read_excel("Amarnameh-1400-1403.xlsx")
                    .rename(columns={
    'ردیف': 'row',
    'سال': 'year',
    'کد ژنریک': 'genericCode',
    'نام ژنریک': 'genericName',
    'مولکول دارویی': 'drugMolecule',
    'نام تجاری فرآورده': 'brandName',
    'شرکت تامین کننده': 'supplier',
    'تاریخ شناسه گذاری': 'registrationDate',
    'تولیدی/وارداتی': 'domesticOrImported',
    'پروانه موقت': 'temporaryLicense',
    'قیمت': 'unitPrice',
    'تعداد تامین شده': 'suppliedVolume',
    'ارزش ریالی': 'rialValue'
    }))
pharma_statistics=pharma_statistics[pharma_statistics["year"].isin([1401,1402])]
pharma_database = {
    "pharma_statistics":pharma_statistics.to_dict(orient="records")
}
yearly_analysis = (
    pharma_statistics.groupby("year")
      .agg(sales_value=("rialValue", "sum"),
           sales_volume=("suppliedVolume", "sum"))
      .reset_index()
)
yearly_analysis["value_growth_rate"] = yearly_analysis["sales_value"].pct_change() * 100
yearly_analysis["volume_growth_rate"] = yearly_analysis["sales_volume"].pct_change() * 100
pharma_database["yearly_analysis"] = yearly_analysis.to_dict(orient="records")

<div dir="rtl">

# تحلیل ۱۰ تامین‌کننده برتر در سال ۱۴۰۲

در این بخش، داده‌های ۱۰ تامین‌کننده برتر از نظر **ارزش فروش در سال ۱۴۰۲** آماده شده است.

### مراحل انجام شده:
1. **محاسبه ارزش فروش هر تامین‌کننده** برای سال‌های ۱۴۰۱ و ۱۴۰۲.
2. **محاسبه درصد رشد ارزش فروش** سال ۱۴۰۲ نسبت به ۱۴۰۱.
3. **محاسبه حجم فروش (suppliedVolume)** هر تامین‌کننده برای سال‌های ۱۴۰۱ و ۱۴۰۲.
4. **محاسبه درصد رشد حجم فروش** سال ۱۴۰۲ نسبت به ۱۴۰۱.
5. ترکیب همه اطلاعات در یک DataFrame به نام `supplier_analysis`.

### ستون‌های خروجی:
- `supplier`: نام تامین‌کننده
- `1401_value` و `1402_value`: ارزش فروش در سال‌های ۱۴۰۱ و ۱۴۰۲
- `1402_value_GR`: درصد رشد ارزش فروش سال ۱۴۰۲ نسبت به ۱۴۰۱
- `1401_volume` و `1402_volume`: حجم فروش در سال‌های ۱۴۰۱ و ۱۴۰۲
- `1402_volume_GR`: درصد رشد حجم فروش سال ۱۴۰۲ نسبت به ۱۴۰۱

</div>


In [10]:
top_ten_suppliers_1402 = (
    pharma_statistics.pivot_table(
        index="supplier",
        columns="year",
        values="rialValue",
        aggfunc="sum",
        fill_value=0
    )
      .sort_values(1402, ascending=False).head(10).reset_index())

top_ten_suppliers_1402_value_GR=((top_ten_suppliers_1402.set_index("supplier"))
                                 .pct_change(axis=1).
                                 reset_index()
                                 .drop(columns=[1401]))

supplier_analysis = top_ten_suppliers_1402.rename(columns={
    1401: '1401_value',
    1402: '1402_value',
})
supplier_analysis = supplier_analysis.merge(
    top_ten_suppliers_1402_value_GR,
    how='left',
    on='supplier'
).rename(columns={1402:'1402_value_GR'})
supplier_volume = (
    pharma_statistics.pivot_table(
        index="supplier",
        columns="year",
        values="suppliedVolume",
        aggfunc="sum",
        fill_value=0
    ).reset_index()
)

top_ten_suppliers_1402_volume = supplier_volume[
    supplier_volume['supplier'].isin(top_ten_suppliers_1402['supplier'])].rename(columns={
    1401: '1401_volume',
    1402: '1402_volume',
})
top_ten_suppliers_1402_volume_GR=((top_ten_suppliers_1402_volume.set_index("supplier"))
                                 .pct_change(axis=1)
                                  .reset_index()
                                 .drop(columns=['1401_volume'])
                                  .rename(columns={'1402_volume':'1402_volume_GR'}))
supplier_analysis = supplier_analysis.merge(top_ten_suppliers_1402_volume,how='left',on='supplier')
supplier_analysis=supplier_analysis.merge(top_ten_suppliers_1402_volume_GR, how='left', on='supplier')


pharma_database["supplier_analysis"] = supplier_analysis.to_dict(orient="records")


## 📊 تحلیل سالانه فروش و نرخ رشد

در این بخش، عملکرد فروش سالانه رو با استفاده از نمودارهای ستونی بررسی شد تا روند تغییرات ارزش فروش، حجم فروش و نرخ رشد سال‌به‌سال مشخص بشه.

ابتدا داده‌های سالانه‌ی استخراج‌شده از پایگاه اطلاعات دارویی رو به‌صورت DataFrame آماده شد و مقادیر مورد نیاز مثل ارزش فروش (به تریلیون ریال) و نرخ رشد رو محاسبه کردم.

سپس سه نمودار در کنار هم رسم شد:
- نمودار اول: ارزش فروش در هر سال
- نمودار دوم: حجم فروش
- نمودار سوم: نرخ رشد ارزش و حجم فروش (به درصد)

در نهایت، برای هر نمودار رنگ‌بندی مجزا، عنوان، و برچسب مقادیر روی میله‌ها تنظیم شده تا تحلیل دیداری ساده‌تر و خواناتر بشه.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
yearly_df = pd.DataFrame(pharma_database["yearly_analysis"])
yearly_df["Year"] = yearly_df["year"].astype(str)  # convert year to string for plotting
yearly_df["Sales Value (trillion IRR)"] = round(yearly_df["sales_value"] / 1e12, 2)
yearly_df["Sales Volume"] = yearly_df["sales_volume"]
yearly_df["Value Growth Rate (%)"] = yearly_df["value_growth_rate"]
yearly_df["Volume Growth Rate (%)"] = yearly_df["volume_growth_rate"]

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Sales Value (trillion IRR)", "Sales Volume", "Growth Rate (%)")
)

# Sales Value subplot
fig.add_trace(
    go.Bar(
        x=yearly_df["Year"],
        y=yearly_df["Sales Value (trillion IRR)"],
        text=yearly_df["Sales Value (trillion IRR)"],
        name="Sales Value"
    ),
    row=1, col=1
)
fig.add_trace(
    go.Bar(
        x=yearly_df["Year"],
        y=yearly_df["Sales Volume"],
        text=yearly_df["Sales Volume"],
        name="Sales Volume"
    ),
    row=1, col=2
)
fig.add_trace(
    go.Bar(
        x=yearly_df["Year"],
        y=yearly_df["Value Growth Rate (%)"],
        text=[f"{v:.2f}%" for v in yearly_df["Value Growth Rate (%)"]],
        name="Value Growth Rate"
    ),
    row=1, col=3
)
fig.add_trace(
    go.Bar(
        x=yearly_df["Year"],
        y=yearly_df["Volume Growth Rate (%)"],
        text=[f"{v:.2f}%" for v in yearly_df["Volume Growth Rate (%)"]],
        name="Volume Growth Rate"
    ),
    row=1, col=3
)

fig.update_layout(
    title_text="Annual Sales and Growth Analysis",
    showlegend=True,
    barmode="group"
)

fig.show()
# fig.write_image("annual_analysis.png")

# تحلیل ۱۰ تأمین‌کننده برتر سال ۱۴۰۲

این تحلیل عملکرد ۱۰ تأمین‌کننده برتر در سال ۱۴۰۲ را نشان می‌دهد. معیارهای مورد بررسی شامل **ارزش فروش، حجم فروش** و **نرخ رشد آن‌ها** است.

## نمودارها

1. **ارزش فروش ۱۴۰۲ (ریال):** مجموع ارزش فروش هر تأمین‌کننده در سال ۱۴۰۲.
2. **حجم فروش ۱۴۰۲:** مجموع تعداد یا حجم فروش هر تأمین‌کننده در سال ۱۴۰۲.
3. **نرخ رشد ارزش فروش ۱۴۰۲ (%):** درصد تغییر ارزش فروش نسبت به سال قبل.
4. **نرخ رشد حجم فروش ۱۴۰۲ (%):** درصد تغییر حجم فروش نسبت به سال قبل.

> نمودارها به صورت چهار زیرنمودار در یک تصویر نشان داده شده‌اند:
> - بالای سمت چپ: ارزش فروش
> - بالای سمت راست: حجم فروش
> - پایین سمت چپ: رشد ارزش فروش
> - پایین سمت راست: رشد حجم فروش

**عنوان نمودار اصلی:** تحلیل ۱۰ تأمین‌کننده برتر سال ۱۴۰۲

این نمودار کمک می‌کند تا به راحتی **مقایسه عملکرد تأمین‌کنندگان** و شناسایی **رشد یا کاهش فروش** هر یک در سال ۱۴۰۲ انجام شود.


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

df = supplier_analysis.copy()
suppliers = df['supplier']
value_1402 = df['1402_value']
volume_1402 = df['1402_volume']
value_GR = df['1402_value_GR'] * 100
volume_GR = df['1402_volume_GR'] * 100

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Sales Value 1402 (IRR)",
        "Sales Volume 1402",
        "Sales Value Growth 1402 (%)",
        "Sales Volume Growth 1402 (%)"
    )
)

fig.add_trace(go.Bar(x=suppliers, y=value_1402), row=1, col=1)
fig.add_trace(go.Bar(x=suppliers, y=volume_1402), row=1, col=2)
fig.add_trace(go.Bar(x=suppliers, y=value_GR), row=2, col=1)
fig.add_trace(go.Bar(x=suppliers, y=volume_GR), row=2, col=2)

fig.update_layout(height=500, width=800, title_text="Top 10 Suppliers Analysis 1402", showlegend=False)
fig.show()
# fig.write_image("top10_suppliers_1402.png")
